In [1]:
import sys
import math
import random
import itertools
import os
import numpy as np
import scipy.sparse.csgraph
import networkx as nx
from SetCoverPy import *
from gurobipy import *

In [2]:
INPUT_DIR = './inputs'
input0 = os.path.join(INPUT_DIR, '2.in')
def read_file(file):
    with open(file, 'r') as f:
        data = f.readlines()
    data = [line.strip().split() for line in data]
    return data

def data_parser(input_data):
    number_of_kingdoms = int(input_data[0][0])
    list_of_kingdom_names = input_data[1]
    starting_kingdom = input_data[2][0]
    adjacency_matrix = [[entry if entry == 'x' else float(entry) for entry in row] for row in input_data[3:]]
    return number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix
def matrix(input_file):
    adj = np.genfromtxt(input_file, dtype=float, skip_header=3)
    size = len(adj)
    for i in range(size):
        for j in range(size):
            if np.isnan(adj[i][j]):
                adj[i][j] = 0
    return adj

def preprocess(input_file):
    data = read_file(input_file)
    number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = data_parser(data)
    adjacency_matrix = matrix(input_file)
    return number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix

number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = preprocess(input0)

In [3]:
g = matrix(input0)
gnx =nx.from_numpy_matrix(g)
paths = list(nx.all_pairs_dijkstra_path(gnx))
recon = []
for p in paths:
    a = []
    for i in range(number_of_kingdoms):
        a.append(p[1][i])
    recon.append(a)
shortest_dist = scipy.sparse.csgraph.floyd_warshall(g)

In [4]:
conquer_cost = np.diag(g)
conquer_cost

array([10.,  5.,  1.,  5.,  5.])

In [5]:
m = Model()

edge_vars = {}
vertex_vars = {}
n = shortest_dist.shape[0] #number of kingdoms



# Utils
def get_neighbors_and_self(i):
    if conquer_cost[i] == 0:
        return np.nonzero(g[i])[0].tolist() + [i]
    else:
        return np.nonzero(g[i])[0].tolist()
    
def is_graph_covered_by(tour):
    my_set = set()
    for i in tour:
        for j in get_neighbors_and_self(i):
            my_set.add(j)
    return my_set == set(range(n))
    
    
def subtourelim(model, where):
    if where == GRB.callback.MIPSOL:
        selected = []
        # make a list of edges selected in the solution
        for i in range(n):
            sol = model.cbGetSolution([model._vars[i,j] for j in range(n)])
            selected += [(i,j) for j in range(n) if sol[j] > 0.5]
        print(selected,'selected')
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        print(tour,'tour')
        if 0 not in tour or (not is_graph_covered_by(tour)):
            # add a subtour elimination constraint
            print('not covered')
            expr = 0
            if len(tour) == 1:
                node = tour[0]
                model.cbLazy(quicksum(model._vars[i, node] for i in range(n)) >= 1)
                model.cbLazy(quicksum(model._vars[node, i] for i in range(n)) >= 1)
            else:
                for i in range(len(tour)):
                    for j in range(i+1, len(tour)):
                        print('fuck you!!!')
                        expr += model._vars[tour[i], tour[j]]
                        print("=========", expr)
                        model.cbLazy(expr <= len(tour) - 1)

# Given a list of edges, finds the shortest subtour

def subtour(edges):
  visited = [False]*n
  cycles = []
  lengths = []
  selected = [[] for i in range(n)]
  for x,y in edges:
    selected[x].append(y)
  while True:
    current = visited.index(False)
    thiscycle = [current]
    while True:
      visited[current] = True
      neighbors = [x for x in selected[current] if not visited[x]]
      if len(neighbors) == 0:
        break
      current = neighbors[0]
      thiscycle.append(current)
    cycles.append(thiscycle)
    lengths.append(len(thiscycle))
    if sum(lengths) == n:
      break
  return cycles[lengths.index(min(lengths))]

In [6]:
n

5

In [7]:
g

array([[10.,  6., 20.,  0.,  0.],
       [ 6.,  5., 14.,  0.,  0.],
       [20., 14.,  1.,  3.,  2.],
       [ 0.,  0.,  3.,  5.,  5.],
       [ 0.,  0.,  2.,  5.,  5.]])

In [8]:
edge_vars = {}
# Add edges as variables - Binary Constraint(5)
for i in range(n+1): # +1 to account for dummy node
    for j in range(i+1):
        edge_vars[i,j] = m.addVar(obj=1, vtype=GRB.BINARY, name='is_path_taken_'+str(i)+'_'+str(j))
        edge_vars[j,i] = edge_vars[i,j]
    m.update()

# Add all pair flow variable (except for dummy)
for i in range(n):
    for j in range(i+1):
        edge_vars[i, j, "flow"] = m.addVar(obj=1, vtype=GRB.CONTINUOUS, name='flow_between'+str(i)+'_'+str(j))
        edge_vars[j, i, "flow"] = edge_vars[i, j, "flow"]
        
# Add flow variable j-dummy
for j in range(n):
    edge_vars[j, n, "flow"] = m.addVar(obj=1, vtype=GRB.CONTINUOUS, name='flow_between'+str("_dummy")+'_'+str(j))
    
# Add flow variable dummy-j (Binary) Constraint(11)
for j in range(n):
    edge_vars[n, j, "flow"] = m.addVar(obj=1, vtype=GRB.BINARY, name='flow_between_'+str(j)+'_dummy')


# Add conquer cost Variable
for i in range(n):
    edge_vars[i] = m.addVar(obj=conquer_cost[i], vtype=GRB.BINARY, name='is_conquered_' + str(i))
    m.update()

# Add conquer cost for dummy
edge_vars[n] = m.addVar(obj=2^10, vtype=GRB.BINARY, name='is_conquered_dummy')
    
# Add Constraint (2)
for i in range(n+1):
    if i < n:
        neighbor = get_neighbors_and_self(i)
    if i == n:
        neighbor = set(range(n+1))
    m.addConstr(quicksum(edge_vars[j] for j in neighbor) >= 1)
    m.update()
        
# Add Constraint (3)
for j in range(n+1):
    m.addConstr(quicksum(edge_vars[i,j] for i in range(n+1)) == quicksum(edge_vars[j, k] for k in range(n+1)))
    edge_vars[j,j].ub = 0
    m.update()

# Add Constraint (6)
m.addConstr(quicksum(edge_vars[n, j, "flow"] for j in range(n)) == 1)

# Add Constraint (7)
for j in range(n):
    m.addConstr(quicksum(edge_vars[j, k, "flow"] for k in range(n+1)) == quicksum(edge_vars[i, j] for i in range(n+1)) + quicksum(edge_vars[i, j, "flow"] for i in range(n+1)))

# Constraint (8)
for j in range(n):
    m.addConstr(edge_vars[j, n, "flow"] <= (n+1) * edge_vars[n, j, "flow"])
    
# Constraint (9)
for i in range(n):
    for j in range(n):
        if j != i:
            m.addConstr(edge_vars[i, j, "flow"] <= n * edge_vars[i, j])

# (10)
for i in range(n):
    for j in range(n):
        if j != i:
            m.addConstr(edge_vars[i, j, "flow"] >= 0)


start = list_of_kingdom_names.index(starting_kingdom)
# Add Starting Point Constraint FIXME
# m.addConstr(quicksum(edge_vars[i, start] for i in range(n)) >= 1)
# m.addConstr(quicksum(edge_vars[start, i] for i in range(n)) >= 1)

m.setObjective(quicksum(quicksum(shortest_dist[i,j]*edge_vars[i,j] for i in range(n)) for j in range(n)) 
               + quicksum(edge_vars[i] for i in range(n+1)), GRB.MINIMIZE)

m.update()
m._vars = edge_vars
# m.params.LazyConstraints = 1
m.optimize()

conquered = m.getAttr('x', edge_vars)
# selected = [(i,j) for i in range(n) for j in range(n) if solution[i,j] > 0.5]
print(conquered)

Optimize a model with 63 rows, 52 columns and 138 nonzeros
Variable types: 20 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 63 rows and 52 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
{(0, 0): 0.0, (1, 0): 0.0, (0, 1): 0.0, (1, 1): 0.0, (2, 0): 0.0, (0, 2): 0.0, (2, 1): 0.0, (1, 2): 0.0, (2, 2): 0.0, (3, 0): 0.0, (0, 3): 0.0, (3, 1): 0.0, (1, 3): 0.0, (3, 2): 0.0, (2, 3): 0.0, (3, 3): 0.0, (4, 0): 0.0, (0, 4): 0.0, (4, 1): 0.0, (1, 4): 0.0, (4, 2): 0.0, (2, 4): 0.0, (4, 3): 0.0, (3, 4): 0.0, (4, 4): 0.0, (5, 0)

In [8]:
# Add edges as variables
edge_vars = {}
for i in range(n):
    for j in range(i+1):
        edge_vars[i,j] = m.addVar(obj=1, vtype=GRB.BINARY, name='is_path_taken_'+str(i)+'_'+str(j))
        edge_vars[j,i] = edge_vars[i,j]
    m.update()

# Add is_conquered Variable
for i in range(n):
    edge_vars[i] = m.addVar(obj=conquer_cost[i], vtype=GRB.BINARY, name='is_conquered_' + str(i))
    m.update()
# Add Constraint (2) FIXME
for i in range(n):
    neighbor = get_neighbors_and_self(i)
    m.addConstr(quicksum(edge_vars[j] for j in neighbor) >= 1)
    m.update()
        
# Add Constraint (3), (5)
for i in range(n):
    m.addConstr(quicksum(edge_vars[i,j] for j in range(n)) == quicksum(edge_vars[k, i] for k in range(n)))
    edge_vars[i,i].ub = 0
    m.update()

start = list_of_kingdom_names.index(starting_kingdom)
# Add Starting Point Constraint FIXME
m.addConstr(quicksum(edge_vars[i, start] for i in range(n)) >= 1)
m.addConstr(quicksum(edge_vars[start, i] for i in range(n)) >= 1)

m.update()
m.setObjective(quicksum(quicksum(shortest_dist[i,j]*edge_vars[i,j] for i in range(n)) for j in range(n)) 
               + quicksum(edge_vars[i] for i in range(n)) , GRB.MINIMIZE)
m._vars = edge_vars
m.params.LazyConstraints = 1
m.optimize(subtourelim)
# conquered = m.getAttr('x', edge_vars)
# selected = [(i,j) for i in range(n) for j in range(n) if solution[i,j] > 0.5]
# print(conquered)

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 12 rows, 20 columns and 27 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
[(0, 4), (4, 0)] selected
[1] tour
not covered
[(0, 4), (4, 0)] selected
[1] tour
not covered
[(0, 4), (4, 0)] selected
[1] tour
not covered
[(0, 4), (4, 0)] selected
[1] tour
not covered
Presolve removed 9 rows and 5 columns
Presolve time: 0.00s
Presolved: 3 rows, 15 columns, 10 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)
[(0, 1), (1, 0)] selected
[2] tour
not covered

Root relaxation: objective 1.700000e+01, 4 iterations, 0.00 seconds
[(0, 1), (1, 0), (2, 4), (4, 2)] selected
[3] tour
not covered
[(0, 1), (1, 0), (2, 3), (3, 2)] selected
[4] tour
not covered
[(0, 1), (1, 0), (2, 3), (2, 4), (3, 2), (4, 2)] selected